In [1]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from pandas import read_csv
import numpy as np
import time
import operator
from getData import getData
from getConfig import getConfig
from editConfig import editConfig
from getETSparms import getETSparms
from calcMAPE import calcMAPE

In [2]:
def buildModel(train, m):
    '''
    Build a Holt-Winters ETS model with passed parameters
    '''
    t = m["trend"]
    d = m["damped"]
    s = m["seasonal"]
    p = m["periods"]
    return ExponentialSmoothing(\
                                train,\
                                trend=t,\
                                damped=d,\
                                seasonal=s,\
                                seasonal_periods=p)

In [3]:
def fitModel(model, f):
    '''
    Fit the model
    '''
    b = f["BoxCox"]
    r = f["RemoveBias"]
    alpha = f["smoothing_level"]
    beta = f["smoothing_slope"]
    gamma = f["smoothing_seasonal"]
    
    return model.fit(\
                     optimized=True,\
                     use_boxcox=b,\
                     remove_bias=r,\
                     smoothing_level=alpha,\
                     smoothing_slope=beta,\
                     smoothing_seasonal=gamma)

In [4]:
def predict(model, config):
    '''
    Using the input model, make predictions using "forecast"
    '''
    steps = config["testSize"]
    preds = model.forecast(steps=steps)
    return preds

In [5]:
def saveModel(m, f, aicc, error):
    '''
    Store the results: the parameters used to build the model; various "fit" attributes; the fitted values; the error
    '''
    t = m["trend"] 
    d = m["damped"]
    s = m["seasonal"]
    p = m["periods"]
    b = f["BoxCox"]
    r = f["RemoveBias"]
    alpha = f["smoothing_level"]
    beta = f["smoothing_slope"]
    gamma = f["smoothing_seasonal"]
    aicc = str(round(aicc, 2))
    error = str(round(error,3))
    
    rec = str(t)+","+str(d)+","+str(s)+","+str(p)+","+str(b)+","+str(r)+","+str(alpha)+","+str(beta)+\
    ","+str(gamma)+","+aicc+","+error+"\n"
    summary.write(rec)

In [6]:
def runParms(train, test, m, f, config):
    '''
    For each parameter combination, build a model; fit the model; make predictions; calculate the error
    Each combination is tested repeatedly against a rolling window of Train/Test data
    '''
    try:
        model = buildModel(train, m)
        model = fitModel(model, f)
        aicc = round(model.aicc, 2)
        preds = predict(model, config)
        error = calcMAPE(test, preds)
    except:
        aicc = error = None
    return aicc, error

In [7]:
def gridSearch(ts, modelParms, fitParms, config):
    '''
    Loop through all the parameter combinations
    '''
    numCombos = len(modelParms) * len(fitParms)
    trainWindow = config["trainingSize"]
    testWindow = config["testSize"]
    
    count = 0
    for m in modelParms:
        for f in fitParms:
            errorList = []
            aiccList = []
            for x in range(config["numForecasts"]):
                train = ts[x:(x + trainWindow)]
                test = ts[(x + trainWindow):(x + trainWindow + testWindow)]
                train = np.array(train)
                test = np.array(test)
                aicc, error = runParms(train, test, m, f, config)
                if error is not None:
                    errorList.append(error)
                    aiccList.append(aicc)
            if len(errorList) > 0:
                error = sum(errorList)/len(errorList)
                aicc = sum(aiccList)/len(aiccList)
                saveModel(m, f, aicc, error)
            count += 1
            if count %100 == 0:
                print("Done with forecast {} of {}".format(count, numCombos))
    return

In [8]:
config = getConfig()
editConfig()

df = getData(config)
ts = df[config["labelColumn"]]

modelParms, fitParms = getETSparms()
start = time.time()
with open(config["dataLoc"]+"summary.csv", "w") as summary:
    hdr = "trend"+","+"damped"+","+"seasonal"+","+"periods"+","+"BoxCox"+","+"bias"+\
   ","+"alpha"+ ","+"beta"+ ","+"gamma"+ ","+"aicc"+ ","+"error"+"\n"
    summary.write(hdr)
    gridSearch(ts, modelParms, fitParms, config)

print("Finished after {:.0f} minutes".format((time.time() - start)/60))

Finished after 0 minutes
